# Road Following - Live demo(resnet18デモ)

このノートブックでは、JetBotがコース上を走行することを確認できます。

### Load Trained Model(学習済みモデルの読み込み)

``train_model.ipynb``ノートブックの指示に従って、すでに``best_steering_model_xy.pth``を作成していることを想定します。

PyTorchモデルを読み込んでJetBot用に変更します。これはトレーニング用のnotebookでもおなじみですね。

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

``train_model.ipynb``で学習した``best_steering_model_xy.pth``から学習の重みづけをロードします。

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

デフォルトではモデルのweightはCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
print("before: {}".format(model.fc.weight.type()))
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()
print("after: {}".format(model.fc.weight.type()))

### Creating the Pre-Processing Function(前処理作成)

モデルを読み込みましたが、まだ少し問題があります。  
学習時の入力画像フォーマットと、OpenCVのカメラ画像フォーマットは一致しません。  
これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像(HWC layout)からTensor(CHW layout)に変換します。
2. トレーニング中に使ったのと同じパラメーターを使用して正規化します。カメラから取得した画像データにはRGBの値があり、1ピクセルはRGBをそれぞれint型で[0、255]の範囲で表したものになります。モデルの入力に使う画像データのRGBはfloat型で[0.0、1.0]の範囲になるため、カメラ画像のRGBをそれぞれ255で割る必要があります。
3. カメラ画像をGPUメモリに転送します。
4. 入力画像データをバッチ配列に変更します。学習時に複数の画像を入力に持っているため、予測時にも1枚の画像であっても入力データは配列にする必要があります。

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

# この値はpytorch ImageNetの学習に使われた正規化のパラメータです。
# カメラ画像はこの値で正規化することが望ましいでしょう。
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    # torchvision.transforms.functional.to_tensor(image)はPIL画像をTensor形式に変換します。つまりHWCをCHWに変換します。
    # これをto(device)でGPUメモリに転送とhalf()でfloat16に変換します。
    image = transforms.functional.to_tensor(image).to(device).half()
    # pytorch ImageNetのパラメータと同じ値を使って[0.0-1.0]の範囲に正規化します。
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    # バッチ配列化した画像データを返します。
    return image[None, ...]

すばらしい、これでカメラ画像をニューラルネットワークの入力フォーマットに変換するための、pre-processing関数を定義できました。　

次はカメラを使うので、一度カメラ用のデーモンを再起動しておきます。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

カメラを起動し、画面に表示します。カメラの起動は、大分なれてきましたね。

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=21)

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

モーターの制御に必要なrobotインスタンスを生成します。

In [ ]:
from jetbot import Robot

robot = Robot()

 JetBotの動作パラメータを設定するためのsliderを作成します。
> メモ: テスト時にうまく動作した値でスライダー値を初期化していますが、あなたのデータセットではうまく機能しない可能性があります。そのため、必要に応じてスライダーを増減してセットアップしてください。

1. ``speed gain``：スピードコントローラー。この値を増やすとJetBotのモーター出力が増加します。
2. ``steering gain``：ステアリングゲインコントローラー。JetBotが左右にブレて不安定な場合は、スムーズに走るようにこの値を減らす必要があります。
3. ``steering dgain``：ステアリングディゲインコントローラー。JetBotが左右にブレて不安定な場合は、スムーズに走るようにこの値を増やす必要があります（元の角度を保つパラメータ）
4. ``steering bias``：ステアリングバイアスコントローラー。JetBotがコースの右端または左端に偏って走行する場合は、JetBotが中央のラインまたはコースをたどるまでこのスライダーの値を調整します。ここでは、モーターバイアスと同様にカメラのoffsetも考慮します。

> 注：JetBotの道路追従動作をスムーズにする必要するために、上記のスライダーを少しずつ調整します。

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.3, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

次に、JetBotがどう判断しているかを表示するためにいくつかのsliderを画面に表示します。x, y sliderは、推論されたx, yの値が表示されます。

``steering_slider``は推定したステアリング値が表示されます。この値はターゲットの実際の角度ではなく、ほぼ比例した単純な値です。  
値が``0``の場合は真っ直ぐ進むことを意味します。

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

次は、カメラの画像が変わるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラ画像をPre-processingにかけてモデル入力データに変換する
2. モデル推論の実行
3. ステアリング値を計算する
4. 比例/微分制御（PD）を使用してモーターを制御する

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラと連動して処理する必要があります。

JetBotでは、traitletsライブラリをもちいてCameraクラスを実装することで実現しています。

## JetBotを動かしてみよう
次のコードで``start jetbot``ボタンと``stop jetbot``ボタンを作成します。  
``start jetbot``ボタンを押すとモデルの初期化が実行され、JetBotが動作し始めます。  
``speed gain``を0から少しずつ値を大きくし、前進させます。  
``steering gain``でハンドルの切れ角を調整します。0に近いほど、切れ角がゆるくなります。  
最初の1フレームの実行時にメモリの初期化が実行されるので、ディープラーニングではどんなモデルも最初の1フレームの処理はすこし時間がかかります。

In [ ]:
import ipywidgets
import time

model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

def start_model(c):
    execute({'new': camera.value})
    camera.observe(execute, names='value')
model_start_button.on_click(start_model)
    
def stop_model(c):
    camera.unobserve(execute, names='value')
    time.sleep(1)
    robot.stop()
model_stop_button.on_click(stop_model)

model_widget = ipywidgets.VBox([
    ipywidgets.HBox([speed_gain_slider, steering_gain_slider]),
    ipywidgets.HBox([image_widget]),
    steering_slider,
    ipywidgets.HBox([model_start_button, model_stop_button])
])

display(model_widget)

やったね！JetBotが動作している場合、新しいカメラフレームごとに画像変換、モデル推論、JetBot制御処理が実行されています。

あなたがデータセットを作ったコース上にJetBotを置いてください。そしてコース上を走行するかどうか確認してください。

### Conclusion
以上がライブデモです。今はJetBotがコース上を走行しているのではないでしょうか？　

road followingが上手く行かない場合、正しく走行できるように失敗しやすい場所でさらにデータを追加してください。  
このようにうまくいかない場所を中心にデータを収集すれば、JetBotはさらによく動作するはずです。

## 次はTensorRTに変換してみましょう
ResNet18モデルはTensorRTに変換することができます。  
trtフォルダの中の``convert_to_trt.ipynb``を実行し、学習済みモデルをTensorRT形式に変換し、``live_demo_trt.ipynb``を実行し自動走行します。  
ノートブックメニューから`Kernel`->`Restert Kernel`を選んでJupyter kernelを再起動するか、JetBotを一度再起動してから次に進むとスムーズに進行できます。

[trt/convert_to_trt.ipynb](./trt/convert_to_trt.ipynb) をクリックし、移動します。